In [34]:
import sys
sys.path.append(sys.path[0][:sys.path[0].find('DVML-P7') + len('DVML-P7')])
from Code.UtilityFunctions.run_query import run_query
from SPARQLWrapper import SPARQLWrapper, JSON
from Code.UtilityFunctions.wikidata_functions import wikidata_query
from Code.UtilityFunctions.get_data_path import get_path
import pandas as pd

In [35]:
def dbpedia_query(sparql_query: str):
    """
    It takes a SPARQL query as a string, and returns a pandas dataframe of the results
    
    :param sparql_query: the query you want to run
    :type sparql_query: str
    :return: The query returns the wikidata item id, the wikidata item label, the wikidata item
    description, and the wikidata item category.
    """
    user_agent = "Yelp knowledge graph mapping/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper("https://dbpedia.org/sparql", agent=user_agent)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.json_normalize(results['results']['bindings'])
    return results_df

### How many people live in the city of top ten most prevelent cities in Yelp?

### Wikidata

In [36]:
biz = pd.read_json(get_path("yelp_academic_dataset_business.json"), lines=True)

In [37]:
biz[['city', 'state']].value_counts().head(10)

city           state
Philadelphia   PA       14567
Tucson         AZ        9249
Tampa          FL        9048
Indianapolis   IN        7540
Nashville      TN        6968
New Orleans    LA        6208
Reno           NV        5932
Edmonton       AB        5054
Saint Louis    MO        4827
Santa Barbara  CA        3829
dtype: int64

In [38]:
wikidata_city_dict = {
    'Philadelphia': 'Q1345',
    'Tucson': 'Q18575',
    'Tampa': 'Q49255',
    'Indianapolis': 'Q6346',
    'Nashville': 'Q23197',
    'New Orleans': 'Q34404',
    'Reno': 'Q49225',
    'Edmonton': 'Q2096',
    'Saint Louis': 'Q38022',
    'Santa Barbara': 'Q159288'
    }

In [39]:
list_of_cities = "wd:Q1345 wd:Q18575 wd:Q49255 wd:Q6346 wd:Q23197 wd:Q34404 wd:Q49225 wd:Q2096 wd:Q38022 wd:Q159288"
sparql_query = f"""
SELECT ?city ?population ?cityLabel 
WHERE {{
  ?city p:P1082 ?statement .
  ?statement ps:P1082 ?population .
  ?statement pq:P585 ?date .
  FILTER NOT EXISTS {{
    ?city p:P1082/pq:P585 ?date2 .
    FILTER(?date2 > ?date)
  }}
  VALUES ?city {{{list_of_cities}}} .
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en" . }}
}}
"""

wikidata_pop = wikidata_query(sparql_query=sparql_query)

### DBpedia

In [40]:
dbpedia_city_dict = {
    'Philidelphia': '"Philadelphia"',
    'Tucson': '"Tucson, Arizona"',
    'Tampa': '"Tampa, Florida"',
    'Indianapolis': '"Indianapolis"',
    'Nashville': '"Nashville, Tennessee"',
    'New Orleans': '"New Orleans"',
    'Reno': '"Reno, Nevada"',
    'Edmonton': '"Edmonton"',
    'Saint Louis': '"St. Louis"',
    'Santa Barbara': '"Santa Barbara, California"@en'}

In [41]:
"@en ".join([i for i in dbpedia_city_dict.values()])

'"Philadelphia"@en "Tucson, Arizona"@en "Tampa, Florida"@en "Indianapolis"@en "Nashville, Tennessee"@en "New Orleans"@en "Reno, Nevada"@en "Edmonton"@en "St. Louis"@en "Santa Barbara, California"@en'

In [42]:
# list_of_cities = '"Philadelphia"@en "Tucson, Arizona"@en "Tampa"@en "Indianapolis"@en "Nashville"@en "New Orleans"@en "Reno"@en "Edmonton"@en "Saint Louis"@en "Santa Barbara"@en'
list_of_cities = "@en ".join([i for i in dbpedia_city_dict.values()])
query = f"""
SELECT DISTINCT ?city ?population ?cityname
WHERE {{
    ?city a dbo:City .
    ?city dbo:populationTotal ?population .
    ?city rdfs:label ?cityname .
    VALUES ?cityname{{{list_of_cities}}}
}}
"""
dbpedia_pop = dbpedia_query(query)

### compare WikiData and DBpedia

In [43]:
wikidata_pop[['cityLabel.value', 'population.value']].sort_values(by='population.value', ascending=False)

,cityLabel.value,population.value
3,Indianapolis,887642
0,Santa Barbara,88665
5,Nashville,684410
4,Tucson,542629
9,Tampa,384959
6,New Orleans,383997
7,St. Louis,301578
8,Reno,264165
1,Philadelphia,1603797
2,Edmonton,1010899


In [44]:
dbpedia_pop[['cityname.value', 'population.value']].sort_values(by='population.value', ascending=False)

,cityname.value,population.value
9,Indianapolis,887642
0,"Santa Barbara, California",88665
3,"Nashville, Tennessee",715884
4,"Tucson, Arizona",542629
1,"Tampa, Florida",384959
5,New Orleans,383997
8,St. Louis,301578
2,"Reno, Nevada",264165
6,Philadelphia,1603797
7,Edmonton,1010899


## CQ 2: How many of 10 random cities from the yelp dataset are in both Wikidata and DBpedia?
### How many of these cities have a population?

### Wikidata

In [57]:
city_sample = biz[['city','state']].drop_duplicates().sample(10, random_state=42)
city_sample

,city,state
90691,Edgemont,PA
699,Safety Harbor,FL
95702,Folsom,NJ
136,Land O Lakes,FL
955,Fort Washington,PA
6538,Avondale,LA
4226,Willingboro,NJ
1145,Glen Carbon,IL
943,Mount Laurel,NJ
90,Plainfield,IN


In [123]:
sample_dict = city_sample.set_index('city')['state'].to_dict()

In [146]:
state_update = {'PA': 'Pennsylvania',
'FL': 'Florida',
'NJ': 'New Jersey',
'LA': 'Louisiana',
'IL': 'Illinois',
'IN': 'Indiana'}

sample_dict_updated = {}
# Iterate through the key-value pairs in the first dictionary
for key, value in sample_dict.items():
    # Update the value in the first dictionary with the value from the second dictionary
    sample_dict_updated[key] = state_update[value]
sample_dict_updated["Land O' Lakes"] = sample_dict_updated.pop('Land O Lakes')

sample_dict_updated

{'Edgemont': 'Pennsylvania',
 'Safety Harbor': 'Florida',
 'Folsom': 'New Jersey',
 'Fort Washington': 'Pennsylvania',
 'Avondale': 'Louisiana',
 'Willingboro': 'New Jersey',
 'Glen Carbon': 'Illinois',
 'Mount Laurel': 'New Jersey',
 'Plainfield': 'Indiana',
 "Land O' Lakes": 'Florida'}

In [147]:
df = pd.DataFrame()
for key, value in sample_dict_updated.items():
    sparql_query = f"""
    SELECT DISTINCT ?val ?val2 ?city ?cityLabel ?location ?locationLabel ?instanceOf
    WHERE{{
    VALUES ?val {{"{key}"@en}}
    VALUES ?val2 {{"{value}"@en}}
    ?city ?label ?val.
    ?city rdfs:label ?cityLabel.
    ?city wdt:P131/wdt:P131 | wdt:P131 ?location .
    ?city wdt:P31/wdt:P279* ?instanceOf .
    ?location rdfs:label ?locationLabel .
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }} 
    FILTER(STRSTARTS(?cityLabel, ?val)).
    FILTER(STRSTARTS(?locationLabel, ?val2)).
    FILTER(?instanceOf = wd:Q486972).
    }}
    LIMIT 1
    """
    df2 = wikidata_query(sparql_query=sparql_query)
    df = pd.concat([df, df2], ignore_index=True)
df[['val.value', 'val2.value', 'city.value', 'cityLabel.value', 'location.value', 'locationLabel.value', 'instanceOf.value']]

,val.value,val2.value,city.value,cityLabel.value,location.value,locationLabel.value,instanceOf.value
0,Edgemont,Pennsylvania,http://www.wikidata.org/entity/Q5337787,Edgemont,http://www.wikidata.org/entity/Q1400,Pennsylvania,http://www.wikidata.org/entity/Q486972
1,Safety Harbor,Florida,http://www.wikidata.org/entity/Q952992,"Safety Harbor, Florida",http://www.wikidata.org/entity/Q812,Florida,http://www.wikidata.org/entity/Q486972
2,Folsom,New Jersey,http://www.wikidata.org/entity/Q1083022,"Folsom, New Jersey",http://www.wikidata.org/entity/Q1408,New Jersey,http://www.wikidata.org/entity/Q486972
3,Fort Washington,Pennsylvania,http://www.wikidata.org/entity/Q1133576,Fort Washington,http://www.wikidata.org/entity/Q1400,Pennsylvania,http://www.wikidata.org/entity/Q486972
4,Avondale,Louisiana,http://www.wikidata.org/entity/Q1994608,Avondale,http://www.wikidata.org/entity/Q1588,Louisiana,http://www.wikidata.org/entity/Q486972
5,Willingboro,New Jersey,http://www.wikidata.org/entity/Q1072819,Willingboro,http://www.wikidata.org/entity/Q1408,New Jersey,http://www.wikidata.org/entity/Q486972
6,Glen Carbon,Illinois,http://www.wikidata.org/entity/Q1375820,Glen Carbon,http://www.wikidata.org/entity/Q1204,Illinois,http://www.wikidata.org/entity/Q486972
7,Mount Laurel,New Jersey,http://www.wikidata.org/entity/Q1072657,Mount Laurel,http://www.wikidata.org/entity/Q1408,New Jersey,http://www.wikidata.org/entity/Q486972
8,Plainfield,Indiana,http://www.wikidata.org/entity/Q986631,Plainfield,http://www.wikidata.org/entity/Q1415,Indiana,http://www.wikidata.org/entity/Q486972
9,Land O' Lakes,Florida,http://www.wikidata.org/entity/Q2375799,Land O' Lakes,http://www.wikidata.org/entity/Q812,Florida,http://www.wikidata.org/entity/Q486972


## DBPedia

In [143]:

df = pd.DataFrame()
for key, value in sample_dict_updated.items():
    sparql_query = f"""
    SELECT ?val ?city ?cityName ?county ?state ?stateName
    WHERE {{
    VALUES ?val {{"{key}"}}
    VALUES ?val2 {{"{value}"}}
    ?city a dbo:City ;
            dbp:name ?cityName  .
            ?county dbo:county ?city .
    ?county dbo:state ?state .
    ?state dbp:name ?stateName .
    FILTER(contains(str(?cityName), ?val) && contains(str(?stateName), ?val2)).
    }}

    """
    df2 = dbpedia_query(sparql_query=sparql_query)
    df = pd.concat([df, df2], ignore_index=True)
df

""


## CQ 3: How many cities are in the 5 most prevelent states/provinces in Yelp?

### Wikidata

In [47]:
yelp_state = biz['state'].value_counts().head(10)
yelp_state

PA    34039
FL    26330
TN    12056
IN    11247
MO    10913
LA     9924
AZ     9912
NJ     8536
NV     7715
AB     5573
Name: state, dtype: int64

In [48]:
wd_state_dict = {
'PA': 'wd:Q1400',
'FL': 'wd:Q812',
'TN': 'wd:Q1509',
'IN': 'wd:Q1415',
'MO': 'wd:Q1581',
'LA': 'wd:Q1588',
'AZ': 'wd:Q816',
'NJ': 'wd:Q1408',
'NV': 'wd:Q1227',
'AB': 'wd:Q1951',
}

In [49]:
wd_state_list = " ".join([v for v in wd_state_dict.values()])
sparql_query = f"""
SELECT DISTINCT ?stateLabel ?numCities
WHERE {{
VALUES ?state {{{wd_state_list}}}
{{?state wdt:P31 wd:Q35657 .}}
UNION
{{?state wdt:P31 wd:Q11828004 .}}

{{ SELECT ?state (COUNT(DISTINCT ?city) as ?numCities)
WHERE
{{
    {{?city wdt:P31/wdt:P279* wd:Q1093829.}}
    UNION
    {{?city wdt:P31/wdt:P279* wd:Q515.}}
    UNION
    {{?city wdt:P31/wdt:P279* wd:Q15127012.}}
    ?city wdt:P131/wdt:P131 ?state .
}}
GROUP BY ?state
}}
SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
}}
"""

wikidata_query(sparql_query=sparql_query)

,numCities.datatype,numCities.type,numCities.value,stateLabel.xml:lang,stateLabel.type,stateLabel.value
0,http://www.w3.org/2001/XMLSchema#integer,literal,4,en,literal,Alberta
1,http://www.w3.org/2001/XMLSchema#integer,literal,387,en,literal,Florida
2,http://www.w3.org/2001/XMLSchema#integer,literal,20,en,literal,Nevada
3,http://www.w3.org/2001/XMLSchema#integer,literal,324,en,literal,Tennessee
4,http://www.w3.org/2001/XMLSchema#integer,literal,645,en,literal,Missouri
5,http://www.w3.org/2001/XMLSchema#integer,literal,491,en,literal,Indiana
6,http://www.w3.org/2001/XMLSchema#integer,literal,88,en,literal,Arizona
7,http://www.w3.org/2001/XMLSchema#integer,literal,111,en,literal,Louisiana
8,http://www.w3.org/2001/XMLSchema#integer,literal,92,en,literal,New Jersey
9,http://www.w3.org/2001/XMLSchema#integer,literal,58,en,literal,Pennsylvania


### DBpedia

In [50]:
# Cities of list entities in DBpedia from https://dbpedia.org/page/Template:Cities_in_the_United_States
sparql_query = """
SELECT (COUNT(DISTINCT ?city) AS ?count) ?var
WHERE {
    ?city a dbo:City .
    ?city dct:subject ?location .
    ?location rdfs:label ?var
    VALUES ?var {
"Cities in Florida"@en  
"Cities in Nevada"@en
"Cities in Tennessee"@en
"Cities in Missouri"@en
"Cities in Indiana"@en
"Cities in Arizona"@en
"Cities in Louisiana"@en
"Cities in New Jersey"@en
"Cities in Pennsylvania"@en
"Cities in Alberta"@en
}
}
"""
dbpedia_query(sparql_query=sparql_query)

,count.type,count.datatype,count.value,var.type,var.xml:lang,var.value
0,typed-literal,http://www.w3.org/2001/XMLSchema#integer,57,literal,en,Cities in Pennsylvania
1,typed-literal,http://www.w3.org/2001/XMLSchema#integer,267,literal,en,Cities in Florida
2,typed-literal,http://www.w3.org/2001/XMLSchema#integer,19,literal,en,Cities in Alberta
3,typed-literal,http://www.w3.org/2001/XMLSchema#integer,118,literal,en,Cities in Indiana
4,typed-literal,http://www.w3.org/2001/XMLSchema#integer,20,literal,en,Cities in Nevada
5,typed-literal,http://www.w3.org/2001/XMLSchema#integer,181,literal,en,Cities in Tennessee
6,typed-literal,http://www.w3.org/2001/XMLSchema#integer,671,literal,en,Cities in Missouri
7,typed-literal,http://www.w3.org/2001/XMLSchema#integer,70,literal,en,Cities in Louisiana
8,typed-literal,http://www.w3.org/2001/XMLSchema#integer,47,literal,en,Cities in Arizona


## CQ 4: What drinks exist that are beverages?
## TODO: check more categories

In [51]:
sparql_query = """
SELECT ?value ?valueLabel (COUNT (DISTINCT ?beverage) AS ?count)
    WHERE {
        VALUES ?value {wd:Q40050 wd:Q2095}
        ?beverage wdt:P279+ ?value .
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    GROUP BY ?value ?valueLabel
"""
wikidata_query(sparql_query=sparql_query)[['value.value', 'valueLabel.value', 'count.value']]

,value.value,valueLabel.value,count.value
0,http://www.wikidata.org/entity/Q2095,food,26741
1,http://www.wikidata.org/entity/Q40050,drink,6661


In [52]:
sparql_query = """
SELECT ?value (COUNT (DISTINCT ?beverage) AS ?count)
WHERE {
    VALUES ?value {dbo:Beverage dbo:Food}
    ?beverage rdf:type ?value
}
GROUP BY ?value
"""
dbpedia_query(sparql_query=sparql_query)[['value.value', 'count.value']]

,value.value,count.value
0,http://dbpedia.org/ontology/Beverage,1884
1,http://dbpedia.org/ontology/Food,13111
